## 1.可迭代对象

在Python里面，只要类型对象实现了_iter_方法（该方法能够返回一个迭代对象），那么它的实例对象就被称为可迭代对象(Iterable)，比如字符串、元组、列表、字典、集合等等。而整数、浮点数，由于其类型对象没有实现_iter_，所以它们不是可迭代对象。

可迭代对象的一大特点就是它可以使用for循环进行遍历

在python中可以调用isinstance进行判断：isinstance() 函数来判断一个对象是否是一个已知的类型

In [3]:
from typing import Iterable

print(
    isinstance("", Iterable),
    isinstance((), Iterable),
    isinstance([], Iterable),
    isinstance({}, Iterable),
    isinstance(set(), Iterable),
)  
print(
    isinstance(0, Iterable),
    isinstance(0.0, Iterable),
)  

True True True True True
False False


## 2.迭代器

调用可迭代对象的_iter_方法，得到的就是迭代器。

In [3]:
#不同类型的对象，都有自己的迭代器，举个栗子
lst = [1, 2, 3]
#底层调用的其实是list.__iter__(lst)
it = lst.__iter__()
print(it) 

In [4]:
next(it)

1

In [5]:
next(it)

2

把一个类作为一个迭代器使用需要在类中实现两个方法 \__iter__() 与 \__next__() 。

如果你已经了解的面向对象编程，就知道类都有一个构造函数，Python 的构造函数为 \__init__(), 它会在对象初始化的时候执行。

\__iter__() 方法返回一个特殊的迭代器对象， 这个迭代器对象实现了\__next__() 方法并通过 StopIteration 异常标识迭代的完成。

\__next__() 方法（Python 2 里是 next()）会返回下一个迭代器对象。

迭代器也是可迭代对象，只不过迭代器内部的\__iter__返回的还是它本身。

在创建迭代器的时候，我们更常用内置函数 __iter()__ 函数用来生成迭代器

In [17]:
# 创建迭代器对象
list=[1,2,3,4]
it = iter(list)    
print(next(it))   # 输出迭代器的下一个元素
print (next(it))

1
2


## 生成器

在 Python 中，**使用了 yield 的函数被称为生成器（generator）**。

跟普通函数不同的是，**生成器是一个返回迭代器的函数，只能用于迭代操作，更简单点理解生成器就是一个迭代器**。

在调用生成器运行的过程中，每次遇到 yield 时函数会暂停并保存当前所有的运行信息，返回 yield 的值, 并在下一次执行 next() 方法时从当前位置继续运行。

调用一个生成器函数，返回的是一个迭代器对象。

### xrange()

In [37]:
#xrange() 函数用法与 range 完全相同，所不同的是生成的不是一个数组，而是一个生成器。
xrange(0,8)

<generator object xrange at 0x000002C5CA7FDE40>

In [36]:
#实现xrange函数
def xrange(start, stop, step=1): 
    while start < stop: 
        yield start 
        start += step

**每次代码运行到 yield，该函数都会发射出一个值**，然
后当外部代码需求另一个值时，该函数才会继续运行（之前的状态保持不变）并发
射新的值。当该函数运行结束时，一个 StopIteration 异常会被抛出表明该生
成器已经没有更多的值了。

In [38]:
xx=xrange(0,8)
next(xx)

0

In [39]:
next(xx)

1

例子：生成斐波那契数列

In [44]:
#基础版本
def fab(max): 
    n, a, b = 0, 0, 1 
    while n < max: 
        print(b) 
        a, b = b, a + b 
        n = n + 1
fab(5)

1
1
2
3
5


In [54]:
#创建迭代器版本，将一个类作为一个迭代器使用
class Fab(object): 
    def __init__(self, max): 
        self.max = max 
        self.n, self.a, self.b = 0, 0, 1 
 
    def __iter__(self): 
        return self 
 
    def __next__(self): 
        if self.n < self.max: 
            r = self.b 
            self.a, self.b = self.b, self.a + self.b 
            self.n = self.n + 1 
            return r 
        raise StopIteration()

for n in Fab(5): 
    print(n)

1
1
2
3
5


In [56]:
#利用生成器返回迭代器
def fab(max): 
    n, a, b = 0, 0, 1 
    while n < max: 
        yield b      # 使用 yield
        # print b 
        a, b = b, a + b 
        n = n + 1

for n in fab(5): 
    print(n)

1
1
2
3
5


**内存占用：生成器表达式需要的内存远小于列表表达式，原因在于它只处理当前感兴趣的值**

比如假设我们有一个数字的长列表，我们想要知道其中有多少个数字可以被 3 整除


divisible_by_three = len([n for n in list_of_numbers if n % 3 == 0]) 

divisible_by_three = sum((1 for n in list_of_numbers if n % 3 == 0))


生成器在每当遇到一个能被 3 整除的数字时就会发射一个 1，而不是别的数字。对该生成器发射的所有值求和，我们就能得到跟列表表达式一样的结果。

## 补充：生成器的延迟估值

之前提到，生成器之所以能够节约我们的内存是因为它只处理当前感兴趣的值。在
我们计算的任意点，我们都只能访问当前的值，而无法访问数列中的其他元素（这
种算法我们通常称为“单通”或“在线”）。有时候这会令生成器难以被使用，不过
有很多模块和函数可以帮助解决这一问题。